In [1]:
import logging

# Configurations
from configuration.llm_inference_configuration import APILLMConfiguration
from configuration.embedding_inference_configuration import APIEmbeddingModelConfiguration, LocalEmbeddingModelConfiguration
from configuration.nlp_configuration import NLPConfiguration
from configuration.tool_configuration import ToolConfiguration
from configuration.operator_configuration import CoTOperatorConfiguration
# System components
from llm.request_llm import RequestLLM
from embedding.local_embedding import LocalEmbeddingModel
from embedding.request_embedding import RequestEmbeddingModel
from nlp.spacy_nlp import SpacyNLP
from tools.python_code_runner import PythonCodeRunnerTool
from tools.demonstration_sampling import DemonstrationSamplingTool
# Prompt
from prompt.zero_shot import ZeroShotPrompt
from prompt.few_shot import FewShotPrompt
from prompt.user_message import UserMessagePrompt
from prompt.assistant_message import AssistantMessagePrompt
from prompt.tool_message import ToolMessagePrompt
# Memory
from base_classes.memory.memory_atom import AbstractMemoryAtom
from base_classes.memory.memory_block import AbstractMemoryBlock
from base_classes.memory.memory_topic import AbstractMemoryTopic
from base_classes.memory.datatypes.data_item import PromptDataItem
from base_classes.memory.memory_worker import MemoryWorker
# MCP
# from tools.mcp_server
# Operator
from base_classes.operator import AbstractOperator
from operators.cot import CoTOperator
from operators.debate import DebateOperator
from operators.react import ReactOperator

logging.basicConfig(
    level=logging.CRITICAL,
    format='%(asctime)s\t\t%(levelname)s\t%(message)s',
    handlers=[logging.StreamHandler()]
)

all_logger_names = list(logging.root.manager.loggerDict.keys())
if '' not in all_logger_names:
    all_logger_names.append('')

for logger_name in all_logger_names:
    logger = logging.getLogger(logger_name)
    if logger_name.startswith("Orbit"):
        logger.setLevel(logging.DEBUG)
    elif logger_name == '':
        logger.setLevel(logging.CRITICAL)
    else:
        logger.setLevel(logging.CRITICAL)


c:\Users\ADMIN\miniconda3\envs\agentic_workflow\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. System component test

## 1.1. LLM test

### 1.1.1. Azure OpenAI test

In [ ]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = RequestLLM(gpt_4o_azure_configuration)

try:
    azure_gpt_4o_1 = RequestLLM(gpt_4o_azure_configuration)
except Exception as e:
    logging.info("LLM unique ID test successful")
    logging.error(f"Failed to create AzureOpenAILLM instance: {e}")

In [ ]:
sample_zero_shot_message = ZeroShotPrompt(
    [
        {
            "role": "user", 
            "content": "Health check. Say 'hi' to start the conversation."
        }
    ]
)

sample_zero_shot_message_responses = azure_gpt_4o.query(query=sample_zero_shot_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_zero_shot_message_responses))

In [ ]:
sample_few_shots_message = FewShotPrompt(
    [
        {
            "role": "system", 
            "content": "You are NetMind assistant. You task is to answer to the user anything about Viettel Group."
        },
        {
            "role": "user", 
            "content": "Tell me about Viettel Group."
        },
        {
            "role": "assistant", 
            "content": "Viettel Group is a Vietnamese multinational telecommunications company headquartered in Hanoi, Vietnam. It is a state-owned enterprise and operated by the Ministry of Defence. You can find out more about Viettel Group at https://viettel.vn/."
        },
        {
            "role": "user",
            "content": "What is the revenue of Viettel Group? Tell me the list of child companies of Viettel Group?"
        }
    ]
)

sample_few_shots_message_responses = azure_gpt_4o.query(query=sample_few_shots_message.prompt, num_responses=2)
print(azure_gpt_4o.get_response_texts(sample_few_shots_message_responses))

## 1.2. Embedding test

### 1.2.1. Request embedding model test

In [ ]:
text_embedding_ada_002_config = APIEmbeddingModelConfiguration()
text_embedding_ada_002_config.load("configuration/yaml/embedding/text-embedding-ada-002.yaml")
text_embedding_ada_002 = RequestEmbeddingModel(text_embedding_ada_002_config)

In [ ]:
# text_embedding_ada_002.encode("Hello world")
text_embedding_ada_002.similarity("Hello world", "Hello world")

## 1.3. Tool test

### 1.3.1. Python code runner tool

In [ ]:
python_code_runner_config = ToolConfiguration()
python_code_runner_config.load("configuration/yaml/tools/python_code_runner.yaml")
python_code_runner = PythonCodeRunnerTool(python_code_runner_config)

In [ ]:
input_code_list = [
    {
        "code_string": "a = 11\nb = 20\nresult = a + b\n\ndef greet(name):\n return f'Hello, {name}!'\n \nmessage = greet('World')",
        "id": "1"
    },
    {
        "code_string": "def multiply(x, y):\n\treturn x * y\n\nnum1 = 5\nnum2 = 3\nresult = multiply(num1, num2)\n\nprint(f'The product of {num1} and {num2} is {result}.')",
        "id": "2"
    }
]

results = await python_code_runner.execute(input_code_list=input_code_list)
results

## 1.4. NLP model test 

### 1.4.1. SpaCy NLP test

In [ ]:
spacy_nlp_config = NLPConfiguration()
spacy_nlp_config.load("configuration/yaml/nlp/spacy.yaml")
spacy_model = SpacyNLP(spacy_nlp_config)

In [ ]:
spacy_model.lemmatize("Hello, my name is John Doe. I am a software engineer. I love coding and solving problems.")

# 2. Memory test

In [2]:
from test_utils.memory import *

2025-06-20 21:28:52,351		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: f241643a-7070-4f94-86a5-5b4093db71d2 | Memory Atom type: 1; Memory Atom state: 1
2025-06-20 21:28:52,351		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: e60bfa85-86f4-473a-a63f-16a61be1708a | Memory Atom type: 2; Memory Atom state: 1
2025-06-20 21:28:52,352		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: ee26ea7b-5e99-4703-9d1f-f537bee3ea87 | Memory Atom type: 2; Memory Atom state: 1
2025-06-20 21:28:52,354		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: af9790f7-62b4-49ad-a12e-7834945c7bb0 | Memory Atom type: 3; Memory Atom state: 1
2025-06-20 21:28:52,354		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: 02b25087-a09b-48a2-8e3f-15d3687e715e | Memory Atom type: 2; Memory Atom state: 1
2025-06-20 21:28:52,354		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: 641d0c4e-864d-41d3-904c-aafdb6978741 | Memory Atom type: 1; Memory Atom state: 1
2025-06-20 21:28:52,355		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: 863a8b30-738a-48e8-9e82-cdabb56

In [3]:
# Memory Worker Initiation
spacy_nlp_config = NLPConfiguration()
spacy_nlp_config.load("configuration/yaml/nlp/spacy.yaml")
spacy_model = SpacyNLP(spacy_nlp_config)

gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = RequestLLM(gpt_4o_azure_configuration)

text_embedding_ada_002_config = APIEmbeddingModelConfiguration()
text_embedding_ada_002_config.load("configuration/yaml/embedding/text-embedding-ada-002.yaml")
text_embedding_ada_002 = RequestEmbeddingModel(text_embedding_ada_002_config)

memory_worker = MemoryWorker(
    emb_model=text_embedding_ada_002,
    llm=azure_gpt_4o,
    nlp_model=spacy_model
)

2025-06-20 21:28:57,795		DEBUG	[🔧NLPConfiguration] List of NLPConfiguration's properties: dict_keys(['nlp_id', 'library', 'model_name', 'dependencies'])
2025-06-20 21:28:57,796		DEBUG	[🔧NLPConfiguration] List of NLPConfiguration's sensitive properties: []
2025-06-20 21:28:57,799		DEBUG	[🔧SpacyNLP] System Component ID: SYSTEM_COMPONENT | 0 | Component type: <class 'nlp.spacy_nlp.SpacyNLP'>
2025-06-20 21:28:57,800		DEBUG	[🔧SpacyNLP] NLP model ID: NLP | en_core_web_sm
2025-06-20 21:28:58,848		DEBUG	[🔧APILLMConfiguration] List of APILLMConfiguration's properties: dict_keys(['llm_id', 'model.provider', 'model.model_name', 'model.temperature', 'model.max_tokens', 'model.stream', 'cache.enabled', 'cache.cache_expiry', 'llm_api.api_key', 'llm_api.api_base', 'llm_api.api_version', 'llm_api.deployment_name', 'cost.prompt_token_cost', 'cost.response_token_cost', 'retry.max_retries', 'retry.backoff_factor', 'security.encrypted', 'security.trust_strategy'])
2025-06-20 21:28:58,849		DEBUG	[🔧APILLMCo

In [4]:
memory_worker.feature_engineer_for_memory_block(mem_block_1)
memory_worker.feature_engineer_for_memory_block(mem_block_2)
memory_worker.feature_engineer_for_memory_block(mem_block_3)
memory_worker.feature_engineer_for_memory_block(mem_block_4)
memory_worker.feature_engineer_for_memory_block(mem_block_5)
memory_worker.feature_engineer_for_memory_block(mem_block_6)

memory_worker.feature_engineer_for_memory_topic(mem_topic_1)
memory_worker.feature_engineer_for_memory_topic(mem_topic_2)
memory_worker.feature_engineer_for_memory_topic(mem_topic_3)

2025-06-20 21:29:03,057		DEBUG	[🔧MemoryWorker] Keywords for Memblock aa45905e-89f6-4bfa-ba7d-e0af5503ee1b: ['aa45905e-89f6-4bfa-ba7d-e0af5503ee1b', 'f241643a-7070-4f94-86a5-5b4093db71d2', 'e60bfa85-86f4-473a-a63f-16a61be1708a', 'ee26ea7b-5e99-4703-9d1f-f537bee3ea87', 'af9790f7-62b4-49ad-a12e-7834945c7bb0', '02b25087-a09b-48a2-8e3f-15d3687e715e', 'add_sum()', 'print()', 'nprint()', 'memoryblock', 'memoryatom', 'user', 'message', 'sum', 'assistant', 'response', 'order', 'number', 'python', 'code', 'add_sum', 'function', 'variable', 'result', 'execution', 'name', 'argument', '11\\nb', 'result}.")\\nreturn']
2025-06-20 21:29:05,382		DEBUG	[🔧MemoryWorker] Generated embeddings for Memblock aa45905e-89f6-4bfa-ba7d-e0af5503ee1b
2025-06-20 21:29:05,683		DEBUG	[🔧MemoryWorker] Keywords for Memblock 860713b0-974c-4925-8fb1-be685ee0de1f: ['860713b0-974c-4925-8fb1-be685ee0de1f', '641d0c4e-864d-41d3-904c-aafdb6978741', '863a8b30-738a-48e8-9e82-cdabb56df507', 'd78b33da-372d-4143-aa98-dd445e5841b9', '0

In [5]:
new_query = [{
    'role': 'user',
    'content': 'how can I build 10 banana-shape buildings in Hanoi?'
}]
new_mem_atom = AbstractMemoryAtom(data=PromptDataItem(UserMessagePrompt(new_query)))

new_mem_block = AbstractMemoryBlock()
new_mem_block.add_memory_atom(new_mem_atom)

2025-06-20 21:29:16,936		DEBUG	[🔧AbstractMemoryAtom] Memory Atom ID: 0ed3fa7e-0c67-49a6-9f54-7a84c549454f | Memory Atom type: 1; Memory Atom state: 1
2025-06-20 21:29:16,937		DEBUG	[🔧AbstractMemoryBlock] Added memory atom 0ed3fa7e-0c67-49a6-9f54-7a84c549454f to memory block 50c259b0-f1ac-48d2-abdf-31aa9a5cf9dd.


In [6]:
relevant_topics = memory_worker.select_relevant_topics(new_mem_block, mem_stack, top_n=-1)

2025-06-20 21:29:21,681		DEBUG	[🔧SpacyNLP] NLP processed text: ['how', 'can', 'I', 'build', '10', 'banana', '-', 'shape', 'buildings', 'in', 'Hanoi', '?']
2025-06-20 21:29:21,682		DEBUG	[🔧MemoryWorker] query_word_count: 12
2025-06-20 21:29:21,683		DEBUG	[🔧MemoryWorker] Processing topic d28248be-a2cc-4ecb-a0bd-2209338a59fa
2025-06-20 21:29:21,684		DEBUG	[🔧MemoryWorker] Last block of topic d28248be-a2cc-4ecb-a0bd-2209338a59fa: 3256b64f-dded-4eb9-a31d-6861025d9bda
c:\Users\ADMIN\Code\VTNET\agentic_workflow\base_classes\memory\memory_worker.py:417: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  topic_emb_tensor = torch.tensor(topic_emb, dtype=torch.float64)
2025-06-20 21:29:22,941		DEBUG	[🔧MemoryWorker] Rule 3 - topic_scores: [(<base_classes.memory.memory_topic.AbstractMemoryTopic object at 0x0000024354BAA390>, 0.7267801708184775)]
2025-06-2

In [8]:
print(str(relevant_topics[0][0].chain_of_memblocks[-1]))
# len(relevant_topics)

MemoryBlock 7059c46f-f5ec-476c-be39-1a29743e40bf:
	MemoryAtom 2c368782-59b6-4649-a4b0-c4a520caf3bc:
		User message: 
			If I have 10 apples, eat 3, add 7, and then eat 2, how many apples do I have?
	MemoryAtom 12a61daf-6204-4d29-82ae-c6773cf715a8:
		Assistant response: 
			Let's break it down step by step:
			1. Start with 10 apples.
			2. Eat 3: 10 - 3 = 7 apples left.
			3. Add 7: 7 + 7 = 14 apples.

			4. Eat 2: 14 - 2 = 12 apples.
			You have 12 apples.


# 3. Operator

## 3.1. CoT

In [ ]:
gpt_4o_azure_configuration = APILLMConfiguration()
gpt_4o_azure_configuration.load("configuration/yaml/llm/azure_gpt_4o.yaml")
azure_gpt_4o = AzureOpenAILLM(gpt_4o_azure_configuration)

demonstration_sampling_tool_config = ToolConfiguration()
demonstration_sampling_tool_config.load("configuration/yaml/tools/demonstration_sampling.yaml")
demonstration_sampling_tool = DemonstrationSamplingTool(demonstration_sampling_tool_config)

In [ ]:
cot_operator_config = CoTOperatorConfiguration()
cot_operator_config.load("configuration/yaml/operators/cot.yaml")

In [ ]:
cot_operator = CoTOperator(config=cot_operator_config)

In [ ]:
CoTOperator.__dict__

In [ ]:
AzureOpenAILLM.__dict__['logger']